Quick Drawing is a kaggle challenge for classifying the sketch pictures. The total number of category is 340 and 
there are 50M pictures in the dataset. Each catogory has 25,000 pictures. The difficulty is the drawing might be 
incomplete or the drawing does not match the label. We are trying to develop a CNN to solve the issue.   

According to the challenge, there are 4 kinds of dataset. 

    - Raw data (.ndjson)
    - Simplified drawings (.ndjson)
    - Binary data (.bin)
    - Bitmap data (.npy)
Raw data and Simplified drawings are provided in a vector information of strokes. Bitmap data is rendered into 
28 * 28 grayscale bitmap in the numpy format.  

Maybe we could consider the raw data to improve the performance since it inlcudes the sequence information.


In [12]:
import torch
import time
import torch
import os
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
import numpy as np
from torch.utils import data
import QDdata as QD
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import DataLoader
import resnet as RN

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 300

print('using device:', device)


using device: cuda


In [5]:
# The following codes comes from cs231N.
from average_precision import mapk
def mean_ap_torch(y_val,y_pred_score,k):
    y_pred = [torch.sort(c,descending=True)[::-1][1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def mean_ap_3(y_val,y_pred_score,k):
    y_pred = [np.argsort(c)[::-1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def check_accuracy(loader, model):
    
    model.eval()  # set model to evaluation mode
    
    meanAP = []
    meanAP3 = []
    with torch.no_grad():
        for i ,(x, y) in enumerate(loader):
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            y_pred = model(x)
            y_val = y.reshape(-1,1)
            y_pred = y_pred.data.cpu().numpy()
            
            mAP = 100 * mean_ap_3(y_val,y_pred,1)
            mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
            meanAP.append(mAP)
            meanAP3.append(mAP3)
            if i > 300:
                break
        
        m1 = np.mean(meanAP)
        m3 = np.mean(meanAP3)
        

        if (m3 > QDtrain["mAP3"] + 1) and (m3 > 20):
            if QDtrain["file"] != "":
                os.remove(QDtrain["file"])
            QDtrain["file"] = str("model_parameter" + str(m3)+str(m1))
            torch.save(model.state_dict(),QDtrain["file"])
            QDtrain["mAP3"] = m3
            QDtrain["mAP"] = m1
        print('Got correct (%.2f,%.2f)' % (m1,m3 ))
        
def check_accuracy_part34(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for i ,(x, y) in enumerate(loader):
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
            if i == 20:
                break
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        
def train(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    
    for e in range(epochs):
        
        for i in range(4):
            train_dataset = QD.QDloadData(no=i)          
            train_loader = DataLoader(dataset=train_dataset, batch_size=128,shuffle=True)
            loader_val = train_loader
            for t, (x, y) in enumerate(train_loader):
                t1 = time.time()
                model.train()  # put model to training mode
                x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
                y = y.to(device=device, dtype=torch.long)

                scores = model(x)
                loss = F.cross_entropy(scores, y)

                # Zero out all of the gradients for the variables which the optimizer
                # will update.
                optimizer.zero_grad()

                # This is the backwards pass: compute the gradient of the loss with
                # respect to each  parameter of the model.
                loss.backward()

                # Actually update the parameters of the model using the gradients
                # computed by the backwards pass.
                optimizer.step()
                t2 = time.time()

                #print(e,t,t2-t1)
                if t % print_every == 0:
                    print('Epoch %d,Iteration %d, loss = %.4f' % (e,t, loss.item()))
                    check_accuracy(loader_val, model)

In [6]:
model = RN.resnet50(num_classes=40)
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(),weight_decay = 1e-5,lr=learning_rate)
QDtrain = {"mAP":0,"mAP3":0,"file":""}

In [ ]:
train(model, optimizer, epochs=10)

1. batch size = 64. count =  340 * 10240.
   Iteration 29400, loss = 0.5610
   Got 64 correct (80.22,85.62)
2. batch size =128 count = 340 *10240 * 2
    Iteration 37800, loss = 0.9535
    Got 128 correct (77.65,83.64)
3. batch size =128 count = 340 *10240 * 2 
    train on 28 * 28 drawing transfered from stroke
    Iteration 45900, loss = 0.9149
    Got correct (57.55,65.98)

Run test samples

In [13]:
def evaluate_test(test_data, model,y):
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        meanAP = []
        meanAP3 = []
        for i ,x in enumerate(test_data):
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            #y = y_label.to(device=device, dtype=torch.long)
            y_pred = model(x)
            y_val = y.reshape(-1,1)
            y_pred = y_pred.data.cpu().numpy()
            
            mAP = 100 * mean_ap_3(y_val,y_pred,1)
            mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
            meanAP.append(mAP)
            meanAP3.append(mAP3)
        
        m1 = np.mean(meanAP)
        m3 = np.mean(meanAP3)
        return (m1,m3)

model = model.to(device=device)
checkpoint = torch.load("model_parameter85.371479.44433")
model.load_state_dict(checkpoint)

class_name = QD.QDloadtest(name = "./data1.npy")
class_name = class_name.class_names

class_accuracy = []

for i,name in enumerate(tqdm(class_name)):
    test_data = QD.QDloadtest(name=os.path.join("../cs230/pic96",name+".npz"),image_size=(96,96))
    test_loader = DataLoader(dataset=test_data, batch_size=128, num_workers=2, shuffle=False)
    y = np.ones(len(test_data))*i
    m1,m3 = evaluate_test(test_loader,model,y)
    print(name,m1,m3)
    class_accuracy.append((name,str(m1),str(m3)))
    
with open('class_accuracy.csv',"w+") as fp:
    writer = csv.writer(fp, delimiter=',')
    writer.writerow(["class", "meanAP1", "meanAP3"])  # write header
    writer.writerows(class_accuracy)


  0%|          | 0/40 [00:00<?, ?it/s]

Total number of items: 120000
Total number of items: 120536


  2%|▎         | 1/40 [00:53<34:58, 53.81s/it]

clock 78.25702 84.84473
Total number of items: 144982


  5%|▌         | 2/40 [01:54<35:28, 56.01s/it]

...

 35%|███▌      | 14/40 [13:21<23:42, 54.72s/it]

star 91.780556 93.99464
Total number of items: 127932


 38%|███▊      | 15/40 [14:16<22:47, 54.70s/it]

crocodile 66.413536 74.761086


KeyboardInterrupt: 

1) 96*96, resnet50, epoch 10, GSD. 75.2%
2) 96*96, resnet50, epoch 10, GSD  79.9% model_parametertensor(91.6667)
3) 96*96, resnet50, epoch 10, Adam model_resnet_92.317_90


* convert simplified test dataset to pixel drawsing

Check the accuracy based on the class.

Create test_simplified.
load compressed(96*96) 1.968482494354248
load non compressed(96*96)  0.31299567222595215
load non compressed(28*28) 0.04715704917907715


In [ ]:
for i in range(1,10):
    data = QD.QDcreateData()
    data.create(start=i*3000,dir_name="../cs230/pic96")
    del data

In [ ]:
del data